<a href="https://colab.research.google.com/github/EbsHirani/FrenchTranslation/blob/master/Oui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
eng = []
fre = []
with open("small_vocab_en.csv", "r") as csvfile:
  csvreader = csv.reader(csvfile)
  for row in csvreader:
    new = "" 
    for i in row:
      new = new + i
    eng.append(new)
with open("small_vocab_fr.csv", "r") as csvfile:
  csvreader = csv.reader(csvfile)
  for row in csvreader:
    new = "" 
    for i in row:
      new = new + i
    fre.append(new)



In [0]:
import pandas as pd
import numpy as np



In [0]:
eng = pd.Series(eng)
fre = pd.Series(fre)

In [9]:
eng.head()

0    new jersey is sometimes quiet during autumn  a...
1    the united states is usually chilly during jul...
2    california is usually quiet during march  and ...
3    the united states is sometimes mild during jun...
4    your least liked fruit is the grape  but my le...
dtype: object

In [10]:
fre.head()

0    new jersey est parfois calme pendant l' automn...
1    les états-unis est généralement froid en juill...
2    california est généralement calme en mars  et ...
3    les états-unis est parfois légère en juin  et ...
4    votre moins aimé fruit est le raisin  mais mon...
dtype: object

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
fre = fre.apply(lambda x:  x.lower()+ " <EOS>")
fre_input = fre.copy().apply(lambda x : "<BOS> " + x )
eng = eng.apply(lambda x: x.lower())

def tokenize(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(data)
  return tokenizer.texts_to_sequences(data), tokenizer
def pad_seq(sequences):
  padded = pad_sequences(sequences, padding = "post")
  return padded

Using TensorFlow backend.


In [0]:
eng_seq, eng_tok = tokenize(eng.values)
fre_input_seq, fre_tok = tokenize(fre_input.values)
fre_output_seq = fre_tok.texts_to_sequences(fre.values)
eng_seq = pad_seq(eng_seq)
# fre_output_seq = pad_seq(fre_output_seq)
fre_input_seq = pad_seq(fre_input_seq)
#fre_output_seq = fre_output_seq.reshape(fre_output_seq.shape[0], fre_output_seq.shape[1], 1) #for sparse categorical crsooentropy
eng_max = eng_seq.shape[1]
fre_max = fre_input_seq.shape[1]
fre_output_seq = pad_sequences(fre_output_seq, padding = "post", maxlen = fre_max)
eng_vocab_len = len(eng_tok.word_index)
fre_vocab_len = len(fre_tok.word_index)
fre_idx = {ids: word for word, ids in fre_tok.word_index.items()}
fre_wordx = {word: ids for word, ids in fre_tok.word_index.items()}
fre_idx[0] = '<PAD>' 


In [0]:
from keras.utils.np_utils import to_categorical
fre_output_seq = to_categorical(fre_output_seq, num_classes = fre_vocab_len+1)

In [65]:
eng_seq.shape

(137860, 15)

In [0]:
import pandas as pd 
import numpy as np
from keras.models import Model

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten, LSTM, Input, Embedding
from keras.layers import BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.utils.np_utils import to_categorical


from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization , MaxPooling2D, Activation
LR = 1e-2
enc_input = Input(shape = (None,))
enc_emb = Embedding(eng_vocab_len+1, 64)(enc_input)
enc_lstm =  LSTM(64, return_state = True)
_ , state_h, state_c = enc_lstm(enc_emb)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None,))
dec_emb = Embedding(fre_vocab_len +1, 64)
dec_emb_out = dec_emb(dec_inputs)
dec_lstm = LSTM(64, return_sequences = True, return_state = True)
dec_out, _, _ = dec_lstm(dec_emb_out, initial_state = enc_states)
dec_soft = Dense(fre_vocab_len+1, activation = "softmax")
dec_out = dec_soft(dec_out)
model = Model([enc_input, dec_inputs], dec_out)

In [0]:
from keras.losses import categorical_crossentropy
model.compile(optimizer = Adam(LR), loss = categorical_crossentropy, metrics = ["accuracy"])


In [0]:
model.summary()

In [110]:
model.fit([eng_seq, fre_input_seq], fre_output_seq, batch_size = 128, epochs = 100,validation_split = 0.05, callbacks =[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7), ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)])

Train on 130967 samples, validate on 6893 samples
Epoch 1/100
130967/130967 [==============================] - 119s 910us/step - loss: 0.0280 - accuracy: 0.9907 - val_loss: 0.0258 - val_accuracy: 0.9918
Epoch 2/100
130967/130967 [==============================] - 118s 904us/step - loss: 0.0267 - accuracy: 0.9910 - val_loss: 0.0313 - val_accuracy: 0.9896
Epoch 3/100
130967/130967 [==============================] - 118s 903us/step - loss: 0.0258 - accuracy: 0.9914 - val_loss: 0.0556 - val_accuracy: 0.9837
Epoch 4/100
130967/130967 [==============================] - 114s 874us/step - loss: 0.0265 - accuracy: 0.9911 - val_loss: 0.0405 - val_accuracy: 0.9879
Epoch 5/100
130967/130967 [==============================] - 119s 912us/step - loss: 0.0258 - accuracy: 0.9913 - val_loss: 0.0245 - val_accuracy: 0.9919
Epoch 6/100
130967/130967 [==============================] - 120s 913us/step - loss: 0.0238 - accuracy: 0.9919 - val_loss: 0.0275 - val_accuracy: 0.9910
Epoch 7/100
130967/130967 [=====

In [0]:
enc_model = Model(enc_input, enc_states)
state_h_in = Input(shape = (64,))
state_c_in  = Input(shape = (64,))
dec_state_in = [state_h_in,state_c_in]
dec_emb_out_pred = dec_emb(dec_inputs) 
dec_out_pred, state_h_out, state_c_out = dec_lstm(dec_emb_out_pred, initial_state = dec_state_in)
dec_out_pred = dec_soft(dec_out_pred)
dec_model = Model([dec_inputs] + dec_state_in, [dec_out_pred, state_h_out, state_c_out])


In [0]:
def predict_french(seq):
  eng_pred = pad_sequences(eng_tok.texts_to_sequences([seq]), maxlen = eng_max, padding = "post")
  dec_init = enc_model.predict(eng_pred)
  dec_word = np.array([[fre_wordx["bos"]]]) #<BOS> and <EOS> are getting tokenized to bos and eos by keras tokenizer, better practice would
                                            #better practice would be to append <BOS> and <EOS> to the numpy array after tokenization
  stop = False
  fre_sentence = ""
  while not stop:
      output = dec_model.predict([dec_word] + dec_init)
      output_seq = output[0]
      new_states = output[1:]
      new_id = np.argmax(output_seq[0,-1,:])
      new_word = fre_idx[new_id]
      

      if (new_word ==  "eos" or len(fre_sentence.split()) == fre_max):
        stop = True
      else: 
        fre_sentence += new_word + " "
      dec_word = np.array([[new_id]]) 
      dec_init = new_states
  return fre_sentence




In [114]:
predict_french("how is the weather")

'comment était le temps dans en octobre dernier '

In [0]:
encoding_model_json = enc_model.to_json()
with open("enc_model.json", "w") as json_file:
    json_file.write(encoding_model_json)
decoding_model_json = dec_model.to_json()
with open("enc_model.json", "w") as json_file:
    json_file.write(decoding_model_json)
enc_model.save_weights("enc_model.h5")
dec_model.save_weights("dec_model.h5")